In [50]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents.tools import BaseTool
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
import json

class GreetingCheckPointInput(BaseModel):
    name: str = Field(..., description='username')

In [69]:

from typing import Optional, Type


llm = ChatOpenAI(model='gpt-3.5-turbo-0613')

def greeting_hello():
    name = input(' >> ')
    return f'Hello {name}'

class GreetingHello(BaseTool):
    name = 'greeting_hello'
    description = "The name of the person to greet"

    def _run(self, name):
        result = greeting_hello(name)
        return result

    def _arun(self, name):
        raise NotImplementedError("This tool does not support async")

    # args_schema: Optional[Type[BaseModel]] = GreetingCheckPointInput



In [70]:
from langchain.tools import format_tool_to_openai_function

tools = [GreetingHello()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [71]:
functions[0]

{'name': 'greeting_hello',
 'description': 'The name of the person to greet',
 'parameters': {'properties': {'__arg1': {'title': '__arg1',
    'type': 'string'}},
  'required': ['__arg1'],
  'type': 'object'}}

In [90]:
from langchain.schema import HumanMessage, AIMessage

ai_message = llm.predict_messages([HumanMessage(content='挨拶をしてくれますか？')], functions=functions)

In [73]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'greeting_hello', 'arguments': '{\n  "__arg1": "こんにちは"\n}'}}, example=False)

In [74]:
ai_message.additional_kwargs['function_call']

{'name': 'greeting_hello', 'arguments': '{\n  "__arg1": "こんにちは"\n}'}

In [84]:
from langchain.schema import FunctionMessage

final_message = llm.predict_messages(
    [
        HumanMessage(content='挨拶をしてくれますか？'),
        ai_message,
        FunctionMessage(name='greeting_hello', content='Hello')
    ],
    functions=functions
)

In [85]:
final_message

AIMessage(content='こんにちは！挨拶をすることができますよ。どのような挨拶をお望みですか？', additional_kwargs={}, example=False)

In [92]:
_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'__arg1': 'こんにちは'}

TypeError: greeting_hello() takes 0 positional arguments but 1 was given